In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import statsmodels.api as sm
import graphviz as gr
from matplotlib import style
import seaborn as sns
from matplotlib import pyplot as plt
style.use("fivethirtyeight")

Note: For all sections, I interpretted the instructions regarding sample size as N=100 for the regressions within each simulation, and N=1000 for the number of Monte Carlo Simulations.

# Setting 1: Randomly Assigned Treatment

Scenario A: Excludes Covariates

Scenario B: Includes Covariates

## Setting 1 DAG

In [ ]:
g = gr.Digraph()
g.edge("D-treatment", "Y")
g.edge("X", "Y")
g

In [ ]:
a = .5
b = 1.2
c = 3
d = 2.5
n = 100

dummy_a_estimates = []
dummy_b_estimates = []

RMSE_A_1 = []
RMSE_B_1 = []


for mc_replication in range(1000):
    
    #D is the randomly assigned treatment 
    D = np.random.randint(2, size=n)
    Dvar = pd.DataFrame(D)
    
    #X1 and X2 are observed covariates that are omitted in alternate B.
    X1 = np.random.uniform(0, 1, n)
    X1var = pd.DataFrame(X1)
    X2 = np.random.uniform(0, 15, n)
    X2var = pd.DataFrame(X2)
    
    #Generated error dist.
    e = np.random.normal(0, 10, n)

    #Generated Observed Y values with observed parameters a,b,c,d
    Y = a + b * X1 + c * X2 + d * D + e


    X = pd.concat((X1var, X2var, Dvar), axis=1)
    X.columns = ('X1', 'X2', 'D')
    
    #a represents model w/o x variables. b represents model including x variables.
    mod_a = sm.OLS(Y,sm.add_constant(D))
    mod_b = sm.OLS(Y, sm.add_constant(X))
    res_a = mod_a.fit()
    res_b = mod_b.fit()
    Y_a_hat = res_a.params[0] + res_a.params[1] * D
    Y_b_hat = res_b.params['const'] + res_b.params['X1'] * X1 + res_b.params['X2'] * X2 + res_b.params['D'] * D

    #Calculating Coef. estimates, Biases, and RMSE
    yi = pd.DataFrame(Y)
    yha = pd.DataFrame(Y_a_hat)
    yhb = pd.DataFrame(Y_b_hat)

    SE_A = ((yi - yha) ** 2)
    MSE_A = SE_A.mean()
    RMSE_A = MSE_A ** 0.5

    SE_B = ((yi - yhb) ** 2)
    MSE_B = SE_B.mean()
    RMSE_B = MSE_B ** 0.5


    dummy_a_estimates += [res_a.params[1]]
    dummy_b_estimates += [res_b.params['D']]

    RMSE_A_1 += [RMSE_A]
    RMSE_B_1 += [RMSE_B]

#Final results --- avg. over 1000 Monte Carlo Simulations

miua = round(np.mean(dummy_a_estimates),3)
miub = round(np.mean(dummy_b_estimates),3)

bias_a = round(miua - d,3)
bias_b = round(miub - d,3)

miu_ra = round(np.mean(RMSE_A_1),3)
miu_rb = round(np.mean(RMSE_B_1),3)

print("Treatment effect in situation A, not controlling for covariates is: " + str(miua) + " with a bias of " + str(bias_a) + '.')
print('\n')
print("Treatment effect in situation B, controlling for covariates is: " + str(miub) + " with a bias of " + str(bias_b) + '.')
print('\n')
print("As we see, omitting covariates will not change the effect of a randomly assigned treatment.")
print('\n')
print('\n')
print("RMSE in situation A, not controlling for covariates is: " + str(miu_ra) + '.')
print('\n')
print("RMSE in situation B, controlling for covariates is: " + str(miu_rb) + '.')
print('\n')
print("However, we see that omitting covariates will lead to higher RMSE.")




A real life example of this first setting (Random assignment treatment) is a common example found in literature of a university or high school offering tutoring programs to students on a random/lottery basis. Since this selection is independent of any characteristics that may determine some outcome Y (test score, graduation, success...), the treatment effect of this program will be unbiased regardless of whether other variables are controlled for.

# Setting 2: Confounder
    
Scenario A: Excludes Confounder

Scenario B: Includes Confounder

## Confounder DAG

In [ ]:
g = gr.Digraph()
g.edge("Confounder: Z", "Y")
g.edge("Confounder: Z", "X1")
g

In [ ]:
a = 3.5
b = 2.5
c = 12
n = 100

coef_a_estimates = []
coef_b_estimates = []

RMSE_A_2 = []
RMSE_B_2 = []

for mc_replication in range(1000):

    #Z variable is the confounder
    Z = np.random.uniform(0, 200, n)
    Zvar = pd.DataFrame(Z)

    #randomly generating error distributions for confounder effect on X1 and Y
    e1 = np.random.normal(0, 8, n)
    e2 = np.random.normal(0, 8, n)
    
    #X1 is a function of Z
    X1 = 5 + 3.5*Z + e1
    X1var = pd.DataFrame(X1)
    
    #Y is a function of X1 and Z
    Y = a + b * X1 + c * Z + e2

    X = pd.concat((X1var, Zvar), axis=1)
    X.columns = ('X1', 'Z')
    
    #Scenario A excludes the confounder Z, Scenario B includes the confounder Z
    mod_a = sm.OLS(Y,sm.add_constant(X1))
    mod_b = sm.OLS(Y, sm.add_constant(X))

    res_a = mod_a.fit()
    res_b = mod_b.fit()

    Y_a_hat = res_a.params[0] + res_a.params[1] * X1
    Y_b_hat = res_b.params['const'] + res_b.params['X1'] * X1 + res_b.params['Z'] * Z

    #Calculating Coef. estimates, Biases, and RMSE
    yi = pd.DataFrame(Y)
    yha = pd.DataFrame(Y_a_hat)
    yhb = pd.DataFrame(Y_b_hat)

    SE_A = ((yi - yha) ** 2)
    MSE_A = SE_A.mean()
    RMSE_A = MSE_A ** 0.5

    SE_B = ((yi - yhb) ** 2)
    MSE_B = SE_B.mean()
    RMSE_B = MSE_B ** 0.5


    coef_a_estimates += [res_a.params[1]]
    coef_b_estimates += [res_b.params['X1']]

    RMSE_A_2 += [RMSE_A]
    RMSE_B_2 += [RMSE_B]
    

#Final results --- avg. over 1000 Monte Carlo Simulations

miua = round(np.mean(coef_a_estimates),3)
miub = round(np.mean(coef_b_estimates),3)

bias_a = round(miua - b,3)
bias_b = round(miub - b,3)

miu_ra = round(np.mean(RMSE_A_2),3)
miu_rb = round(np.mean(RMSE_B_2),3)

print("Effect of X1 in situation A, not controlling for confounder is: " + str(miua) + " with a bias of " + str(bias_a) + '.')
print('\n')
print("Effect of X1 in situation B, controlling for confounder is: " + str(miub) + " with a bias of " + str(bias_b) + '.')
print('\n')
print("As we see, omitting confounder will lead to bias in coefficient estimates.")
print('\n')
print('\n')
print("RMSE in situation A, not controlling for covariates is: " + str(miu_ra) + '.')
print('\n')
print("RMSE in situation B, controlling for covariates is: " + str(miu_rb) + '.')
print('\n')
print("We also see that omitting confounder will lead to higher RMSE.")


A real life example of this second setting with a confounder is any example that has to deal with common omitted variable bias. For example, if we take salary or wages as the dependent variable, and we want to determine the effect of education on wages. A confounder, which may lead to bias if omitted could be ability or intelligence, which drives both education level/success and possibly wages/salary. In this case, if the confounder is omitted, the model will over-estimate the effect of education and lead to bias coefficient estimates.

# Setting 3: Selection Bias in Treatment
Scenario A: Controls for the variable in between the path from cause to effect.

Scenario B: Does not controls for the variable in between the path from cause to effect.

## Selection Bias DAG

In [ ]:
g = gr.Digraph()
g.edge("T", "X")
g.edge("T", "Y")
g.edge("Y", "X")
g

In [ ]:
a = 4
b = 5.5
c = 2
d= 7.5
n = 100

treat_a_estimates = []
treat_b_estimates = []

RMSE_A_3 = []
RMSE_B_3 = []

for mc_replication in range(1000):

    #Randomly generate X1 explanatory Variable
    X1 = np.random.uniform(0, 60, n)
    X1var = pd.DataFrame(X1)

    #Randomly generate Treatment Variable with 70% probability of receiving treatment
    treat = np.random.choice(2, size=n, p=[0.3, 0.7])
    treatvar = pd.DataFrame(treat)
    
    #Generating Error Distribution
    e1 = np.random.normal(0, 8, n)
    e2 = np.random.normal(0, 8, n)

    #Variable in between path
    X2 = 4 + 3.5 * treat + e1
    X2var = pd.DataFrame(X2)

    #Generating Oberved Y variable
    Y = a + b * X1 + c * X2 + d * treat + e2

    #Scenario A includes path variable X2; Scenario B excludes path variable X2.
    
    Xa = pd.concat((X1var, X2var, treatvar), axis=1)
    Xa.columns = ('X1', 'X2', 'treat')

    Xb = pd.concat((X1var, treatvar), axis=1)
    Xb.columns = ('X1', 'treat')

    mod_a = sm.OLS(Y,sm.add_constant(Xa))
    mod_b = sm.OLS(Y, sm.add_constant(Xb))

    res_a = mod_a.fit()
    res_b = mod_b.fit()

    Y_a_hat = res_a.params['const'] + res_a.params['X1'] * X1 + res_a.params['X2'] * X2 + res_a.params['treat'] * treat
    Y_b_hat = res_b.params['const'] + res_b.params['X1'] * X1 + res_b.params['treat'] * treat

    #Calculating Coef. estimates, Biases, and RMSE
    yi = pd.DataFrame(Y)
    yha = pd.DataFrame(Y_a_hat)
    yhb = pd.DataFrame(Y_b_hat)

    SE_A = ((yi - yha) ** 2)
    MSE_A = SE_A.mean()
    RMSE_A = MSE_A ** 0.5

    SE_B = ((yi - yhb) ** 2)
    MSE_B = SE_B.mean()
    RMSE_B = MSE_B ** 0.5


    treat_a_estimates += [res_a.params['treat']]
    treat_b_estimates += [res_b.params['treat']]

    RMSE_A_3 += [RMSE_A]
    RMSE_B_3 += [RMSE_B]
    
    
#Final results --- avg. over 1000 Monte Carlo Simulations

miua = round(np.mean(treat_a_estimates),3)
miub = round(np.mean(treat_b_estimates),3)

bias_a = round(miua - d,3)
bias_b = round(miub - d,3)

miu_ra = round(np.mean(RMSE_A_3),3)
miu_rb = round(np.mean(RMSE_B_3),3)


print("Treatment effect in situation A, controlling for path variable is: " + str(miua) + " with a bias of " + str(bias_a) + '.')
print('\n')
print("Treatment effect in situation B, not controlling for path variable is: " + str(miub) + " with a bias of " + str(bias_b) + '.')
print('\n')
print("As we see, omitting path variable will lead to bias in coefficient estimates.")
print('\n')
print('\n')
print("RMSE in situation A, controlling for covariates is: " + str(miu_ra) + '.')
print('\n')
print("RMSE in situation B, not controlling for covariates is: " + str(miu_rb) + '.')
print('\n')
print("We also see that omitting confounder will lead to higher RMSE.")





A real life example of this setting 3 for selection bias involves any treatment that faces a selection process that may possess inherently bias characteristics in determining some outcome of interest. For example, if we wish to test the effectiveness on some vitamin supplements on overall health. The problem is that there is a selection bias, or a variable in the path from the cause to the effect because it is likely that healthy people will not be in the market for vitamin supplements. Thus, the sample population is likely to be skewed towards being unhealthy, this could mean that the estimated effect of the vitamins may be overestimated or underestimated depending on the scenario. Unhealthy people could see a larger effect due to having more area of improvement, or unhealthy people could see a smaller effect due to unhealthy bodies less likely to adapt healthy ways...